# Topic Model for POTUS Speech Corpus using Gensim

### Imports

In [43]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williamLi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Import stopwords from NLTK

In [29]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

### Import corpus

In [94]:
COLUMNS=['doc_id','date','pres','title','speech']
df = pd.DataFrame(columns=COLUMNS)
PRES = 'obama'
pres_list = "adams arthur bharrison buchanan bush carter cleveland clinton coolidge eisenhower fdroosevelt fillmore ford garfield grant gwbush harding harrison hayes hoover jackson jefferson johnson jqadams kennedy lbjohnson lincoln madison mckinley monroe nixon obama pierce polk reagan roosevelt taft taylor truman tyler vanburen washington wilson".split()

In [91]:
import os
import re
_id = 1
for filename in os.listdir('./speeches'):
    if filename == '.DS_Store':
        continue
    for speech in os.listdir('./speeches/' + filename):
        temp = open('./speeches/' + filename + '/' + speech, 'r', encoding='utf-8').readlines()
        obj = {}
        obj['doc_id'] = _id
        date = re.findall('"([^"]*)"', temp[1])
        obj['date'] = date[0] if len(date) > 0 else None
        obj['pres'] = filename
        obj['title'] = re.findall('"([^"]*)"', temp[0])[0]
        obj['speech']= "".join(temp[2:])
    
        obj = pd.DataFrame(obj, index=[0])
        df = df.append(obj, ignore_index=True)
        _id += 1
df = df.set_index("doc_id")
df.head()

,date,pres,title,speech
doc_id,,,,
1,"August 10, 1927",coolidge,Address at the Opening of Work on Mount Rushmo...,We have come here to dedicate a cornerstone th...
2,"December 8, 1925",coolidge,Third Annual Message,Members of the Congress: In meeting the consti...
3,"December 6, 1923",coolidge,First Annual Message,Since the close of the last Congress the Natio...
4,"October 20, 1925",coolidge,Message Regarding Relationship of Church and S...,"Mr. Moderator, Members Of The Council:\nIt is ..."
5,"March 4, 1925",coolidge,Inaugural Address,\nMy Countrymen:\n\nNo one can contemplate cur...


### Filter for a specific president

In [92]:
docs = df[df['pres'] == PRES]
docs.head()

,date,pres,title,speech
doc_id,,,,
571,"May 15, 2016",obama,Commencement Address at Rutgers University,Hello Rutgers! <Applause.> R-U rah-rah! <Appla...
572,"January 12, 2016",obama,2016 State of the Union Address,"Mr. Speaker, Mr. Vice President, Members of Co..."
573,"August 28, 2008",obama,Acceptance Speech at the Democratic National C...,To Chairman Dean and my great friend Dick Durb...
574,"August 31, 2010",obama,Address on the End of the Combat Mission in Iraq,"Good evening. Tonight, I'd like to talk to you..."
575,"June 15, 2010",obama,Speech on the BP Oil Spill,"Good evening. As we speak, our nation faces a ..."


### Remove bad characters

In [82]:
data = docs.speech.values.tolist()
data = [re.sub('\n+', ' ', sent) for sent in data]
data = [re.sub(r"\<.*\>"," ",sent) for sent in data]

### Tokenize

In [79]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
data_words = list(sent_to_words(data))

print(data_words[:1])

[['hello', 'rutgers']]


### Build the bigram and trigram models

In [80]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['hello', 'rutgers']


### Remove Stopwords, Make Bigrams and Lemmatize

In [60]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [83]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['rutger']]


### Create the Dictionary and Corpus needed for Topic Modeling

In [84]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1)]]


In [63]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ability', 1),
  ('able', 1),
  ('abroad', 1),
  ('accelerate', 1),
  ('access', 2),
  ('accomplish', 2),
  ('accountable', 1),
  ('act', 4),
  ('action', 1),
  ('activist', 1),
  ('activity', 1),
  ('actual', 1),
  ('actually', 2),
  ('adapt', 1),
  ('add', 1),
  ('address', 1),
  ('administration', 1),
  ('administrator', 2),
  ('advocacy', 2),
  ('affordable', 1),
  ('afraid', 1),
  ('afternoon', 1),
  ('age', 1),
  ('ago', 1),
  ('agree', 3),
  ('aid', 1),
  ('air', 1),
  ('alice', 1),
  ('alienate', 1),
  ('alliance', 1),
  ('allow', 1),
  ('ally', 2),
  ('almost', 1),
  ('already', 3),
  ('also', 5),
  ('alter', 2),
  ('always', 7),
  ('america', 14),
  ('american', 10),
  ('amount', 1),
  ('analysis', 1),
  ('anniversary', 1),
  ('answer', 3),
  ('anti', 2),
  ('anything', 1),
  ('anyway', 2),
  ('apart', 1),
  ('apartheid', 1),
  ('apathy', 1),
  ('applause', 76),
  ('approve', 2),
  ('arc', 4),
  ('argument', 1),
  ('array', 1),
  ('art', 1),
  ('asia', 1),
  ('ask', 2),
  

### Build LDA model
Save the model in `./models/`

In [93]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.save('./models/'+PRES+'-model')

### Print the Keywords in the topics
Save them to a txt in `./outputs/`

In [103]:
file = open("./output/"+PRES+".txt","w")
for topic in lda_model.print_topics():
    file.write(str(topic[0]) + ": " + topic[1])
file.close()

doc_lda = lda_model[corpus]

### Compute Model Perplexity and Coherence Score

In [87]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.5030652180508515

Coherence Score:  0.262617810475234


### Visualize the topics

In [88]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.028392  0.019014       1        1  27.680229
1     -0.019189  0.087125       2        1  25.963339
0     -0.045033 -0.024165       3        1  23.404852
3     -0.049145 -0.066756       4        1  22.517897
4      0.141760 -0.015218       5        1   0.433680, topic_info=     Category        Freq                Term       Total  loglift  logprob
3     Default   61.000000            congress   61.000000  30.0000  30.0000
10    Default  162.000000                make  162.000000  29.0000  29.0000
19    Default   75.000000             tonight   75.000000  28.0000  28.0000
2332  Default   57.000000           insurance   57.000000  27.0000  27.0000
633   Default   49.000000             promise   49.000000  26.0000  26.0000
496   Default   60.000000                 man   60.000000  25.0000  25.0000
820   Default  137.000000                time  137.000000  24.0000  24.0000
538   Default   45.000000               never   45.000000  23.0000  23.0000
14    Default   47.000000           president   47.000000  22.0000  22.0000
12    Default   25.000000              member   25.000000  21.0000  21.0000
1700  Default   36.000000              muslim   36.000000  20.0000  20.0000
530   Default   87.000000                must   87.000000  19.0000  19.0000
403   Default   32.000000                idea   32.000000  18.0000  18.0000
604   Default   64.000000                plan   64.000000  17.0000  17.0000
840   Default   13.000000               truth   13.000000  16.0000  16.0000
1924  Default    9.000000                self    9.000000  15.0000  15.0000
378   Default   52.000000         health_care   52.000000  14.0000  14.0000
146   Default   28.000000             citizen   28.000000  13.0000  13.0000
956   Default   42.000000            continue   42.000000  12.0000  12.0000
1301  Default   21.000000               faith   21.000000  11.0000  11.0000
23    Default  164.000000                year  164.000000  10.0000  10.0000
182   Default   30.000000            coverage   30.000000   9.0000   9.0000
532   Default  115.000000              nation  115.000000   8.0000   8.0000
991   Default   13.000000              endure   13.000000   7.0000   7.0000
1119  Default   14.000000              secure   14.000000   6.0000   6.0000
686   Default   82.000000               right   82.000000   5.0000   5.0000
387   Default   34.000000             history   34.000000   4.0000   4.0000
432   Default   86.000000                iraq   86.000000   3.0000   3.0000
425   Default   17.000000  insurance_companie   17.000000   2.0000   2.0000
1533  Default   15.000000             justice   15.000000   1.0000   1.0000
...       ...         ...                 ...         ...      ...      ...
1824   Topic5    0.145591       distinguished    3.580886   2.2381  -6.6424
840    Topic5    0.268162               truth   13.486919   1.5227  -6.0317
3      Topic5    0.394181            congress   61.205719   0.3954  -5.6464
12     Topic5    0.270230              member   25.133896   0.9079  -6.0240
10     Topic5    0.406598                make  162.496521  -0.5500  -5.6154
538    Topic5    0.271846               never   45.692223   0.3162  -6.0180
14     Topic5    0.272205           president   47.406796   0.2807  -6.0167
496    Topic5    0.279769                 man   60.798515   0.0593  -5.9893
19     Topic5    0.275853             tonight   75.293228  -0.1687  -6.0034
820    Topic5    0.296067                time  137.133133  -0.6975  -5.9327
595    Topic5    0.177779              people  225.624008  -1.7055  -6.4427
539    Topic5    0.172423                 new  117.716476  -1.0855  -6.4733
23     Topic5    0.172127                year  164.722763  -1.4232  -6.4750
54     Topic5    0.173306            american  231.912888  -1.7584  -6.4682
530    Topic5    0.163043                must   87.259132  -0.8420  -6.5292
532    